# Lab 07

## Zadanie 1

Przygotuj demo programu znajdującego miejsca zerowe metodą Newtona. Wykorzystując `argparse` ([link](https://docs.python.org/3.6/library/argparse.html#module-argparse)) lub `optparse` ([link](https://docs.python.org/3.6/library/optparse.html)) obsłuż:
- ustalane punktu startowego,
- wielkość kroku w pochodnej,
- ilość kroków metody,
- dokładność
- pomoc

Program uruchamiamy podając, np.:

`./newton.py x**2+x+1 -h 0.00001`

In [1]:
!python3 lab07_files/newton.py x**2-1 -s 7

Found root at 1.00000002018138 after 6 iterations


## Zadanie 2

Zadanie należy wykonać wykorzystując program `BagOfWords` z poprzednich zajęć.


Przerób ją tak, żeby interpunkcja, cyfry i wszelkie inne znaki nie przeszkadzały w parsowaniu tekstu. Uruchom ja na [tekście hamleta](http://www.gutenberg.org/cache/epub/1787/pg1787.txt). Ile razy występuje słowo hamlet? Jak brzmi 10 najczęściej występujących słów?

In [2]:
from collections import defaultdict
import string
import json

class BagOfWords(object):

    def __init__(self, source=None):
        self._dict = defaultdict(int)
        
        if source:
            if isinstance(source, str):
                self.count_words_in_line(source)      
            elif hasattr(source, 'read'):
                for line in source:
                    self.count_words_in_line(line)
            elif isinstance(source, self.__class__):
                self._dict = source._dict.copy()
                
    def count_words_in_line(self, line):
        translator = str.maketrans(string.punctuation + string.digits, " " * 42, "")
        line = line.translate(translator).lower()
        for word in line.split():
            if word.isalpha():
                self._dict[word] += 1
    
    def __contains__(self, word):
        return word in self._dict
    
    def __iter__(self):
        return iter(sorted(self._dict, key=lambda word: self._dict[word], reverse = True))
    
    def __add__(self, other):
        if isinstance(other, self.__class__):
            combined_bag = BagOfWords(self)
            for word, count in other._dict.items():
                combined_bag._dict[word] += count
            return combined_bag
        else:
            return NotImplemented
        
    def __getitem__(self, key):
        if isinstance(key, str):
            key = key.lower()
            if key in self._dict:
                return self._dict[key]
            else:
                raise KeyError(key)
        else:
            raise TypeError

    def __setitem__(self, key, value):
        if isinstance(key, str):
            key = key.lower()
            if isinstance (value, int):
                if key in self._dict:
                    self._dict[key] = value
                else:
                    raise KeyError(key)
            else:
                raise ValueError(value)
        else:
            raise TypeError
      
    def __str__(self):
        return ', '.join(f'{k}:{v}' for k,v in self._dict.items())
    
    def save(self, filename):
        with open(filename, 'w') as file:
            json.dump(self._dict, file)

    def load(self, filename):
        with open(filename, 'r') as file:
            data = json.load(file)
        self._dict = defaultdict(int, data)

In [3]:
import requests

url = "https://www.gutenberg.org/cache/epub/1787/pg1787.txt"

response = requests.get(url)
hamlet_text = response.text

hamlet_bag = BagOfWords(hamlet_text)

In [4]:
print("Liczba wystąpień słowa 'hamlet':", hamlet_bag['hamlet'])

print("10 najczęściej występujących słów:")
for word in list(hamlet_bag)[:10]:
    print(word, hamlet_bag[word])

Liczba wystąpień słowa 'hamlet': 125
10 najczęściej występujących słów:
the 1348
and 1106
to 888
of 887
you 664
a 646
i 631
in 520
my 520
it 447


## Zadanie 3


Wykorzystując `pickle` zapisz i odczytaj klasy z poprzedniego zadania nakarmonej Hamletem. Porównaj metody i rozmiar.

In [5]:
import pickle

with open('lab07_files/hamlet.pickle', 'wb') as file:
    pickle.dump(hamlet_bag, file)

In [6]:
with open('lab07_files/hamlet.pickle', 'rb') as file:
    hamlet_bag2: BagOfWords = pickle.load(file)

print("Liczba wystąpień słowa 'hamlet':", hamlet_bag2['hamlet'])

print("10 najczęściej występujących słów:")
for word in list(hamlet_bag2)[:10]:
    print(word, hamlet_bag2[word])

Liczba wystąpień słowa 'hamlet': 125
10 najczęściej występujących słów:
the 1348
and 1106
to 888
of 887
you 664
a 646
i 631
in 520
my 520
it 447


In [7]:
import sys
print("Original BagOfWords:", sys.getsizeof(hamlet_bag), "bytes")
print("Loaded BagOfWords:", sys.getsizeof(hamlet_bag2), "bytes")

Original BagOfWords: 56 bytes
Loaded BagOfWords: 56 bytes


## Zadanie 4

Wykorzystując [https://gist.github.com/pamelafox/986163](https://gist.github.com/pamelafox/986163) podaj aktualną godzinę we:
- wszystkich krajach, wyświetlając je zgrupowane względem kontynentów,
- przeprowadź symulacyjne wyświetlanie kolejnych krajów w miarę jak w danym kraju wybija północ, opóżnienie wyświetlania ustaw proporcjonalne do realnego czasu

In [8]:
from lab07_files.countryinfo import countries
from datetime import datetime
import pytz
import time

In [9]:
def print_time_info(utc_now):
    for country in sorted(countries,key=lambda x: x["continent"]):
        country_timezone_list = [pytz.timezone(x) for x in country["timezones"]]
        country_times_list = [str(utc_now.replace(tzinfo=pytz.utc).astimezone(x)) for x in country_timezone_list]

        print(f"kontynent: {country['continent']}, kraj: {country['name']} czas: {country_times_list}")
    
print_time_info(datetime.utcnow())

kontynent: Africa, kraj: Angola czas: ['2023-11-13 19:44:12.105710+01:00']
kontynent: Africa, kraj: Burkina Faso czas: ['2023-11-13 18:44:12.105710+00:00']
kontynent: Africa, kraj: Burundi czas: ['2023-11-13 20:44:12.105710+02:00']
kontynent: Africa, kraj: Benin czas: ['2023-11-13 19:44:12.105710+01:00']
kontynent: Africa, kraj: Botswana czas: ['2023-11-13 20:44:12.105710+02:00']
kontynent: Africa, kraj: Democratic Republic of the Congo czas: ['2023-11-13 19:44:12.105710+01:00', '2023-11-13 20:44:12.105710+02:00']
kontynent: Africa, kraj: Republic of the Congo czas: ['2023-11-13 19:44:12.105710+01:00']
kontynent: Africa, kraj: CÃ´te d'Ivoire czas: ['2023-11-13 18:44:12.105710+00:00']
kontynent: Africa, kraj: Cameroon czas: ['2023-11-13 19:44:12.105710+01:00']
kontynent: Africa, kraj: Cape Verde czas: ['2023-11-13 17:44:12.105710-01:00']
kontynent: Africa, kraj: Djibouti czas: ['2023-11-13 21:44:12.105710+03:00']
kontynent: Africa, kraj: Egypt czas: ['2023-11-13 20:44:12.105710+02:00']


In [10]:
def timezone_calculate(timezone, utc_now):
    _time = utc_now.replace(tzinfo=pytz.utc).astimezone(pytz.timezone(timezone))
    return _time.hour * 60 + _time.minute 

def sort_by_timezone():
    utc_now = datetime.utcnow()
    timezone_dict = dict()
    for _dict in countries:
        timezones_list = _dict["timezones"]
        for timezone in timezones_list:
            time_from_timezone = timezone_calculate(timezone, utc_now)
            if time_from_timezone in timezone_dict:
                timezone_dict[time_from_timezone].add(_dict["name"])
            else:
                timezone_dict[time_from_timezone] = {_dict["name"]}
    return sorted(timezone_dict.items(), key=lambda x: -x[0])

data = sort_by_timezone() 

#time.sleep((1440-data[0][0])*60)

#while True:
for x in data:
    print(f"wybiła północ w: {x[1]}")
        #time.sleep(1200)

wybiła północ w: {'Uzbekistan', 'Maldives', 'Pakistan', 'Tajikistan', 'Kazakhstan', 'Turkmenistan', 'Russia'}
wybiła północ w: {'Afghanistan'}
wybiła północ w: {'Mauritius', 'Azerbaijan', 'Georgia', 'Seychelles', 'United Arab Emirates', 'Oman', 'Armenia', 'Russia'}
wybiła północ w: {'Iran'}
wybiła północ w: {'Bahrain', 'Comoros', 'Turkey', 'Somalia', 'Belarus', 'Uganda', 'Kenya', 'Ukraine', 'Eritrea', 'Jordan', 'Djibouti', 'Ethiopia', 'Iraq', 'Kuwait', 'Qatar', 'Saudi Arabia', 'Yemen', 'Russia', 'Tanzania', 'Syria', 'Madagascar'}
wybiła północ w: {'Democratic Republic of the Congo', 'Greece', 'Israel', 'Libya', 'Namibia', 'Egypt', 'Moldova', 'Zimbabwe', 'Romania', 'Finland', 'Malawi', 'South Africa', 'Ukraine', 'Lebanon', 'Cyprus', 'Sudan', 'Mozambique', 'Swaziland', 'Zambia', 'Russia', 'Bulgaria', 'Lesotho', 'Rwanda', 'Latvia', 'Burundi', 'Estonia', 'Botswana', 'Lithuania'}
wybiła północ w: {'Benin', 'Democratic Republic of the Congo', 'Gabon', 'France', 'Norway', 'Algeria', 'Morocco'

## Zadanie 5

Dla klasy `BagOfWords` napisz metody `save` oraz `load` wykorzystujące `json`'a do zapisu i odczytu danych.

In [11]:
url_json = "lab07_files/hamlet_saved.json"

hamlet_bag.save(url_json)

hamlet_loaded_from_json = BagOfWords()
hamlet_loaded_from_json.load(url_json)

print("Liczba wystąpień słowa 'hamlet':", hamlet_loaded_from_json['hamlet'])

print("10 najczęściej występujących słów:")
for word in list(hamlet_loaded_from_json)[:10]:
    print(word, hamlet_loaded_from_json[word])


Liczba wystąpień słowa 'hamlet': 125
10 najczęściej występujących słów:
the 1348
and 1106
to 888
of 887
you 664
a 646
i 631
in 520
my 520
it 447
